POT Simplified Mode tutorial

In [ ]:
import os
import torch
import json
from pathlib import Path

# Set the data and model directories
DATA_DIR = './data'
MODEL_DIR = 'model'

os.makedirs(MODEL_DIR, exist_ok=True)

In the first step we need to export the model, which we need to optimize, into the ONNX format:

In [ ]:
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_resnet20", pretrained=True)
model.eval()
dummy_input = torch.randn(1, 3, 32, 32)

onnx_model_path = Path(MODEL_DIR) / 'cifar10_resnet20.onnx'
ir_model_xml = onnx_model_path.with_suffix('.xml')
ir_model_bin = onnx_model_path.with_suffix('.bin')

torch.onnx.export(model, dummy_input, onnx_model_path)

Now we convert this model into the OpenVINO IR using the Model Optimizer

In [ ]:
!mo --framework=onnx --data_type=FP32 --input_shape=[1,3,32,32] -m $onnx_model_path  --output_dir $MODEL_DIR

In this step we need to write the optimization config and export it to .json file

In [ ]:
config = dict({
    "model": {
        "model_name": "resnet20",
        "model": str(ir_model_xml),
        "weights": str(ir_model_bin)
    },

    "engine": {
        "type": "simplified",
        "data_source": DATA_DIR
    },

    "compression": {
        "target_device": "ANY",
        "algorithms": [
            {
                "name": "DefaultQuantization",
                "params": {
                    "preset": "performance",
                    "stat_subset_size": 300
                }
            }
        ]
    }
})

with open("config.json", "w") as outfile:
    json.dump(config, outfile)

Now we can optimize the model using this configuration file by calling the following command

In [ ]:
!pot -c config.json

In the next releases the following method for using the simplified mode quantization without writing a POT config will be available

In [ ]:
!pot -q default -m $ir_model_xml -w $ir_model_bin --engine=simplified --data-source=$DATA_DIR